<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex02_175480/ex02_175480_Patrick_Ferreira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 2: Análise de Sentimentos usando Bag of Words

Neste notebook iremos treinar um rede de uma única camada para fazer análise de sentimento usando o dataset IMDB.

In [1]:
import numpy as np

nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


# Importando as bibliotecas necessárias

In [2]:
import collections
import pandas as pd
import re
import torch
from typing import List

# Preparando Dados

Primeiro, fazemos download do dataset:

In [3]:
!wget -nc http: // files.fast.ai / data / examples / imdb_sample.tgz
!tar -xzf imdb_sample.tgz

--2022-08-31 00:32:03--  ftp://http/
           => ‘.listing’
Resolving http (http)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘http’
//: Scheme missing.
File ‘index.html’ already there; not retrieving.

/: Scheme missing.
File ‘index.html’ already there; not retrieving.

/: Scheme missing.
File ‘index.html’ already there; not retrieving.

/: Scheme missing.
File ‘index.html’ already there; not retrieving.



Carregamos o dataset .csv usando o pandas:

In [4]:
df = pd.read_csv('imdb_sample/texts.csv')
df.shape
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


Iremos agora apenas selecionar 100 exemplos de treinamento:

In [5]:
treino = df[df['is_valid'] == False]  # Apenas treinamento, isto é, descartamos o dataset de validação.

print('treino.shape original:', treino.shape)

treino = treino[:100]  # Aqui truncamos o dataset para os 100 primeiros exemplos. 

print('treino.shape depois:', treino.shape)

treino.shape original: (800, 3)
treino.shape depois: (100, 3)


Iremos dividir este conjunto em entrada (X) e saída desejada (Y, target) e converter as strings "positive" e "negative" do target para valores booleanos:

In [6]:
X_treino = treino['text']
Y_treino = treino['label']

print(f'Primeiras linhas de X_treino:\n{X_treino.head()}\n')
print(f'Primeiras linhas de Y_treino:\n{Y_treino.head()}\n')

mapeamento = {'positive': True, 'negative': False}
Y_treino = Y_treino.map(mapeamento)
Y_treino = torch.tensor(Y_treino.values, dtype=torch.long)
print(f'Tamanho de Y_treino: {Y_treino.shape}')
print(f'5 primeiras linhas de Y_treino: {Y_treino[:5]}')
print(f'Número de exemplos positivos: {(Y_treino == True).sum()}')
print(f'Número de exemplos negativos: {(Y_treino == False).sum()}')

Primeiras linhas de X_treino:
0    Un-bleeping-believable! Meg Ryan doesn't even ...
1    This is a extremely well-made film. The acting...
2    Every once in a long while a movie will come a...
3    Name just says it all. I watched this movie wi...
4    This movie succeeds at being one of the most u...
Name: text, dtype: object

Primeiras linhas de Y_treino:
0    negative
1    positive
2    negative
3    positive
4    negative
Name: label, dtype: object

Tamanho de Y_treino: torch.Size([100])
5 primeiras linhas de Y_treino: tensor([0, 1, 0, 1, 0])
Número de exemplos positivos: 51
Número de exemplos negativos: 49


# Definindo o tokenizador

Agora temos a função de tokenização, isto é, que converte strings para tokens.

In [7]:
import re
import string


def tokenize(text: str):
    """
    Convert string to a list of tokens (i.e., words).
    This function lower cases everything and removes punctuation.
    """

    return re.sub('[' + string.punctuation + ']', '', text).lower().split()

## Testando a função com um exemplo simples


In [8]:
assert tokenize("?I? lik.e, t\'o ea!t pizza.") == ['i', 'like', 'to', 'eat', 'pizza'], "Não passou no assert."
print('Passou no assert!')

Passou no assert!


# Definindo o vocabulário

Selecionaremos os `max_tokens` (ex: 1000) tokens mais frequentes do dataset de treino como sendo nosso vocabulário.

In [9]:
from collections import Counter


def create_vocab(texts: List[str], max_tokens: int):
    """
    Returns a dictionary whose keys are tokens and values are token ids (from 0 to max_tokens - 1).
    """

    tokens = []

    for t in texts:
        tokens.extend(tokenize(t))

    return dict(Counter(tokens).most_common(max_tokens))

## Testando a função


In [10]:
L = ['f', 'a', 'a', 'd', 'b', 'd', 'c', 'e', 'a', 'b', 'e', 'e', 'a', 'd']
k = 3
resultado = create_vocab(L, k)

assert f'resultado: {resultado}' == "resultado: {'a': 4, 'd': 3, 'e': 3}"
print("Passou no assert!")

Passou no assert!


# Função para converter string para Bag-of-words

In [11]:
def convert_to_bow(text: str, vocab):
    """
    Returns a bag-of-word vector of size len(vocab).
    """
    bow = np.zeros(len(vocab))
    vocab_words = list(vocab.keys())

    for token in tokenize(text):
        if token in vocab_words:
            bow[vocab_words.index(token)] = 1

    return torch.tensor(bow)

## Testando a função

In [12]:
X_assert = ["How much wood would a woodchuck chuck if a woodchuck could chuck wood?",
            "Peter Piper picked a peck of pickled peppers. How many pickled peppers did Peter Piper pick?",
            "She saw Sharif's shoes on the sofa. But was she so sure those were Sharif's shoes she saw?", ]

vocab = create_vocab(X_assert, max_tokens=1000)

bow = convert_to_bow(X_assert[0], vocab)

assert bow.tolist() == [1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
print("Passou no Assert!")

Passou no Assert!


## Definindo a Rede Neural

**Entrada:**

$x \in R^{B \times |V|}$     (bag-of-words)

**Parametros:**

$W \in R^{|V| \times K}$    (weights: matriz de pesos)

$b \in R^{K}$    (bias/viés)

**Saída:**

$p \in R^{B \times K}$  (probabilidade de cada classe)


**Onde:**

$K$ = número de classes

$B$ = tamanho do batch

$|V|$ = tamanho do vocabulário

**Definição da rede:**

$z = xW + b$   (camada linear. $z$ é chamado de logits)

$p_i = \frac{e^{z_i}}{\sum_{j=0}^{K-1} e^{z_j}}$   (softmax)



In [13]:
class MyModel():

    def __init__(self, dim: int):
        self.weights = (2 * torch.randn((dim, 2), dtype=torch.double) - 1) * 0.01
        self.bias = (2 * torch.randn((2,), dtype=torch.double) - 1) * 0.01

        self.weights.requires_grad = self.bias.requires_grad = True

    def __call__(self, x):
        network_output = torch.matmul(x.double(), self.weights) - self.bias
        probs = torch.softmax(network_output, -1)
        return probs

## Testando modelo com uma entrada aleatória

Escreva abaixo um pequeno código para testar se seu modelo processa uma matriz de entrada de tamanho `batch_size, dim`, ou seja, a matriz contém `batch_size` exemplos, cada um sendo representado por um vetor de tamanho `dim`.

In [14]:
batch_size = 16
dim = 8
model = MyModel(dim)
x = torch.randn(batch_size, dim)
probs = model(x)

print(probs)

tensor([[0.5197, 0.4803],
        [0.5139, 0.4861],
        [0.5079, 0.4921],
        [0.5351, 0.4649],
        [0.5565, 0.4435],
        [0.5102, 0.4898],
        [0.5167, 0.4833],
        [0.5158, 0.4842],
        [0.5294, 0.4706],
        [0.5219, 0.4781],
        [0.5116, 0.4884],
        [0.5177, 0.4823],
        [0.5642, 0.4358],
        [0.4925, 0.5075],
        [0.5047, 0.4953],
        [0.5065, 0.4935]], dtype=torch.float64, grad_fn=<SoftmaxBackward>)


# Função de custo Entropia Cruzada

$y \in R^{K}$  (target),

a equação da entropia cruzada associada a um exemplo é dada por:

$L = \sum_{i=0}^{K-1} -y_i \log p_i$   (esta é a loss por exemplo)

Se $y$ for um vetor one-hot (apenas um dos elementos é diferente de zero), podemos simplicar a equação acima para:

$L = -\log p_i$

Onde $i$ é o indice da classe correta. Ou seja, $p_i$ é a probabilidade que o modelo colocou na classe correta.

A função de custo é a **média** da entropia cruzada de cada exemplo no batch.

In [15]:
def cross_entropy_loss(probs, targets):
    """
    Args:
      probs: a float32 matrix of shape (batch_size, number of classes)
      targets: a long (int64) array of shape (batch_size)

    Returns:
      Mean loss in the batch.
    """
    # Rescreva o código abaixo sem usar laço.
    # batch_size = probs.shape[0]
    # losses = []
    # for i in range(batch_size):
    #     losses.append(-torch.log(probs[i, targets[i]]))
    #
    # losses = torch.stack(losses)
    targets_one_hot = torch.nn.functional.one_hot(targets, num_classes=2)
    return (targets_one_hot * probs).sum(axis=-1).mean()

## Testando a função entropia cruzada com probabilidades de 50%

Escreva abaixo um pequeno código para testar se a entropia cruzada confere com a resposta do problema 3.6 do exercício da semana passada. Crie um tensor para as probabilidades (50%) e um target também aleatório balanceado e calcule a cross entropia. Qual é o valor esperado da cross entropia nesse caso?

In [16]:
# escreva seu código aqui

# Convertendo dataset de treino para uma matriz de bag-of-words

In [17]:
vocab = create_vocab(X_treino, max_tokens=1000)
bows = []
for text in X_treino:
    bow = convert_to_bow(text, vocab)
    bows.append(bow)

X = torch.stack(bows)
print(X.shape)

torch.Size([100, 1000])


# Laço de Treinamento

In [18]:
num_iterations = 100
learning_rate = 1.5

model = MyModel(dim=len(vocab))

for i in range(num_iterations):
    # Zera os gradientes
    if model.weights.grad is not None:
        model.weights.grad.data.zero_()
        model.bias.grad.data.zero_()

    probs = model(X)
    loss = cross_entropy_loss(probs, Y_treino)
    print(f'iteration: {i}  loss: {loss:.6f}  exp(loss): {torch.exp(loss):.4f}')
    loss.backward()

    #Atualiza os pesos
    model.weights.data = model.weights.data - learning_rate * model.weights.grad.data
    model.bias.data = model.bias.data - learning_rate * model.bias.grad.data

iteration: 0  loss: 0.487749  exp(loss): 1.6286
iteration: 1  loss: 0.356911  exp(loss): 1.4289
iteration: 2  loss: 0.418367  exp(loss): 1.5195
iteration: 3  loss: 0.378514  exp(loss): 1.4601
iteration: 4  loss: 0.424852  exp(loss): 1.5294
iteration: 5  loss: 0.224874  exp(loss): 1.2522
iteration: 6  loss: 0.449845  exp(loss): 1.5681
iteration: 7  loss: 0.223017  exp(loss): 1.2498
iteration: 8  loss: 0.447841  exp(loss): 1.5649
iteration: 9  loss: 0.250297  exp(loss): 1.2844
iteration: 10  loss: 0.391258  exp(loss): 1.4788
iteration: 11  loss: 0.139870  exp(loss): 1.1501
iteration: 12  loss: 0.246465  exp(loss): 1.2795
iteration: 13  loss: 0.298586  exp(loss): 1.3480
iteration: 14  loss: 0.149915  exp(loss): 1.1617
iteration: 15  loss: 0.164458  exp(loss): 1.1788
iteration: 16  loss: 0.191188  exp(loss): 1.2107
iteration: 17  loss: 0.158788  exp(loss): 1.1721
iteration: 18  loss: 0.139160  exp(loss): 1.1493
iteration: 19  loss: 0.087306  exp(loss): 1.0912
iteration: 20  loss: 0.063822 